In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees,
    [0] - long,
    [1] - lat)
    """
    x1, y1, x2, y2 = map(np.radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = np.sin(dy/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dx/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km

In [ ]:
plt.style.use('Solarize_Light2')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [12, 15]

srcData = "G:/1_Data1/sea/AllSea.txt"
destData = "G:/1_Data1/sea/AllSeaGisOut.txt"
obrisData = "G:/1_Data1/obris/obris.dat"
#obrisWest = "G:/1_Data1/obris/obrisWest.dat"
#obrisEast = "G:/1_Data1/obris/obrisEast.dat"
batimetrFile = "G:/1_Data1/batimetria/b1.txt"
# srcData = "~/methaneSea.dat

In [ ]:
df = pd.read_csv("G:/1_Data1/sea/AllSea.txt",
                 header=0,
                 usecols=['Longitude', 'Latitude', 'DateTime', 'cCH4Wtr', 'cCO2Wtr'],
                 # skip_blank_lines=True,
                 skipinitialspace=True,
                 na_values='--',
                 skiprows=[1,2,3,4],
                 sep='\t',
                 decimal=',',
                 #nrows=5000,
                 parse_dates=['DateTime'],
                 dayfirst=True
                 )

In [ ]:
print(f"size before {df.shape}")
df.dropna(axis='index', subset=['Longitude', 'Latitude', 'cCH4Wtr', 'cCO2Wtr'], inplace=True)
print(f"size after {df.shape}")

In [ ]:
df['year'] = df['DateTime'].dt.year

In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
for yr in range(2013, 2024):
    t = df.loc[(df['year'] == yr) & df['cCH4Wtr']].shape
    print(f'{yr}: {t[0]}')

In [ ]:
obris = pd.read_csv("G:/1_Data1/obris/obris.dat",
                        header=0,
                        na_values='--',
                        sep='\t',
                        decimal=',',
                       )

In [ ]:
obrisEast = pd.read_csv("G:/1_Data1/obris/obrisEast.dat",
                        header=0,
                        na_values='--',
                        sep='\t',
                        decimal=',',
                       )

In [ ]:
obrisWest = pd.read_csv("G:/1_Data1/obris/obrisWest.dat",
                        header=0,
                        na_values='--',
                        sep='\t',
                        decimal=',',
                       )
obrisWest.head(3)

In [ ]:
fig, ax = plt.subplots()
ax.plot(obrisEast['long'], obrisEast['lat'], 'b-')
ax.plot(obrisWest['long'], obrisWest['lat'], 'r-')


In [ ]:
longStep = 0.01
latStep = 0.005

df['long'] = (df['Longitude']/longStep).round()*longStep
df['lat'] = (df['Latitude']/latStep).round()*latStep
df['year'] = df['DateTime'].dt.year

In [ ]:
#func_list = ['mean', 'count']
func_list = ['mean']
cols_to_calc = {'cCH4Wtr': func_list,
                'cCO2Wtr': func_list,
               }
res = df.groupby(['long', 'lat'], as_index=False).agg(cols_to_calc)
res

## GIS handlers   
### Get Squared Baikal

In [ ]:
nordPoint = (109.767, 55.751)
sudPoint = (103.709166666667, 51.7079166666667)

In [ ]:
longr = np.radians(res['long'])
latr = np.radians(res['lat'])
obrisWest['longr'] = np.radians(obrisWest['long'])
obrisWest['latr'] = np.radians(obrisWest['lat'])
obrisEast['longr'] = np.radians(obrisEast['long'])
obrisEast['latr'] = np.radians(obrisEast['lat'])

In [ ]:
for i in range(len(res)):
    argg = (float(res.loc[i, 'long']), float(res.loc[i, 'lat']))
    res.at[i, 'toWest'] = obrisWest.apply(lambda x: dist((x['long'], x['lat']), argg), axis=1).min()
    res.at[i, 'toEast'] = obrisEast.apply(lambda x: dist((x['long'], x['lat']), argg), axis=1).min()
    res.at[i, 'toSud'] = dist(sudPoint, argg)
    res.at[i, 'toNord'] = dist(nordPoint, argg)
    print('*', end='')
    

In [ ]:
res['norm_x'] = res['toWest']/(res['toWest'] + res['toEast'])
res['norm_y'] = res['toSud']/(res['toSud'] + res['toNord'])
res

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
fig, ax = plt.subplots(1, 2)
ax[0].scatter(res['long'], res['lat'], s=30, c=np.log(res[('cCO2Wtr', 'mean')]), marker='o', linewidth=0, alpha=0.1)
ax[1].scatter(res['norm_x'], res['norm_y'], s=30, c=np.log(res[('cCO2Wtr', 'mean')]), marker='o', linewidth=0, alpha=0.1)

In [ ]:
res.to_csv('all_gases_dist.dat',
           sep='\t'
          )

In [ ]:
%matplotlib inline
valMax = res[('cCO2Wtr','mean')].max

fig, ax = plt.subplots()
ax.set_xlim(106, 108)
ax.set_ylim(52, 53)

# wether this or those
ax.scatter(res['long'], res['lat'], c=res[('cCO2Wtr', 'mean')], s=40, marker='o', linewidth=0, cmap='rainbow', alpha=0.1)
ax.plot(obris['long'], obris['lat'], 'b-')

fig.savefig('gis.png')

In [ ]:
tomsk = (85.1974, 56.372)
moscow = (37.929, 55.422)

dist(tomsk, moscow)

In [ ]:
dist((0, 89), (1, 89))

In [ ]:
from scipy import spatial

In [ ]:
# This solution by @denis is OK for small xy2
res['mindistWest'] = np.min(spatial.distance.cdist(res.loc[:, ['long', 'lat']], obrisWest.loc[:, ['long', 'lat']]), axis=1)
res['mindistEast'] = np.min(spatial.distance.cdist(res.loc[:, ['long', 'lat']], obrisEast.loc[:, ['long', 'lat']]), axis=1)
res['norm_xx'] = res['mindistWest']/(res['mindistWest'] + res['mindistEast'])
# res['norm_yy'] = res['toSud']/(res['toSud'] + res['toNord'])

res

In [ ]:
from shapely.geometry import Point
from shapely.geometry import Polygon

In [ ]:
l = list(obrisWest['long']) + list(obrisEast['long'])

In [ ]:
ob = zip(list(obrisWest['long'])+(list(obrisEast['long'])) , list(obrisWest['lat'])+(list(obrisEast['lat'])))
print(ob)

In [ ]:
obrisPolygon = Polygon(ob)

In [ ]:
point = Point(103, 55)

In [ ]:
obrisPolygon.contains(point)

In [ ]:
dff = pd.DataFrame(columns=['long', 'lat'])
dff

#### fill baikal obris with cells `longStep` * `latStep`  
do it once. do no forget save to file

In [ ]:
longStep = 0.01
latStep = 0.005
n=0
for x in np.arange(103.5, 110.0, longStep):
    for y in np.arange(51.4, 55.9, latStep):
        if obrisPolygon.contains(Point(x, y)):
            toWest = obrisWest.apply(lambda f: dist((f['long'], f['lat']), (x, y)), axis=1).min()
            toEast = obrisEast.apply(lambda f: dist((f['long'], f['lat']), (x, y)), axis=1).min()
            toSud = dist(sudPoint, (x, y))
            toNord = dist(nordPoint, (x, y))
            dff.loc[len(dff), ['long', 'lat', 'toWest', 'toEast', 'toSud', 'toNord']] = [x, y, toWest, toEast, toSud, toNord]
            n+=1
            if n%100 == 0:
                print(f"n={n},", end=' ')
print("done!")

In [ ]:
dff['norm_x'] = np.int32(dff['toWest']/(dff['toWest'] + dff['toEast'])*100)
dff['norm_y'] = np.int32(dff['toSud']/(dff['toSud'] + dff['toNord'])*100)
dff

In [ ]:
dff.plot('long', 'lat', c=dff['norm_x'], kind='scatter',  )

In [ ]:
dff.to_csv('wholeBaikalGrid.dat',
          sep='\t')

In [ ]:
surferData = pd.read_csv('out_VAR.dat', sep=' ', header=0, names=['norm_x', 'norm_y', 'cCH4wtr'])
surferData['norm_x'] = np.int32(surferData['norm_x']*100)
surferData['norm_y'] = np.int32(surferData['norm_y']*100)
surferData

In [ ]:
dff = pd.merge(dff, surferData, how='left', on=['norm_x', 'norm_y'])

In [ ]:
dff

In [ ]:
dff.plot('long', 'lat', c=np.log(dff['cCH4wtr']), kind='scatter' )